In [4]:
from bs4 import BeautifulSoup
from requests import get
from pathlib import Path


GRADUS_LINK = "https://gradus.kefo.hu/archive/"
MTMT_LINK = "https://m2.mtmt.hu/gui2/"


# Data
data_json = []


gradus_soup = BeautifulSoup(get(GRADUS_LINK).text, "html.parser")


# Get issues div
issues_tag = gradus_soup.find("div", id="issues")
year_tags = gradus_soup.select('div[style="float: left; width: 100%;"]')


for year_tag in year_tags:

    # Get publication year
    publication_year = year_tag.find("h3", recursive=False).text.strip()

    for volume_info_tag in year_tag.find_all("a"):
        # Split text
        volume_info = volume_info_tag.text.split(":")[0].split("(")[0]
        # Remove unwanted substrings
        volume_info = (
            volume_info.replace("Vol ", "").replace("No ", "").replace(" ", "")
        )

        volume, number = volume_info.split(",")

        # Get volume catalog tag
        volume_catalog_link = GRADUS_LINK + volume_info_tag["href"]
        volume_catalog_soup = BeautifulSoup(
            get(volume_catalog_link).text, "html.parser"
        )
        volume_catalog = volume_catalog_soup.find("div", id="content")

        section = ""

        # Get articles
        for catalog_item in list(
            volume_catalog.select(
                "div#content > .tocSectionTitle, div#content > .tocArticle"
            )
        ):
            if catalog_item.attrs.get("class") == ["tocSectionTitle"]:
                section = catalog_item.text.strip()

            else:
                authors, doi_link = (
                    catalog_item.find(class_="tocAuthors").text.strip().split("\n")
                )

                doi_link_works: bool = get(doi_link).status_code == 200

                pdf_link = f"{volume_catalog_link}/{catalog_item.find('a')['href']}"

                

                data_json.append(
                    {
                        "Publication Year": publication_year,
                        "Volume": volume,
                        "Number": number,
                        "Section": section,
                        "Authors": authors,
                        "DOI Link": doi_link,
                        "DOI Link Status": doi_link_works,
                    }
                )

               

https://gradus.kefo.hu/archive/2023-2/2023_2_ECO_001_Valocikova.pdf
https://gradus.kefo.hu/archive/2023-2/2023_2_ECO_002_Kasznar.pdf
https://gradus.kefo.hu/archive/2023-2/2023_2_ECO_003_Paldi.pdf
https://gradus.kefo.hu/archive/2023-2/2023_2_ECO_004_Seidner.pdf
https://gradus.kefo.hu/archive/2023-2/2023_2_ECO_005_Fenyvesi.pdf


KeyboardInterrupt: 